In [63]:
import pandas as pd
import numpy as np

pd.options.display.float_format = '{:.5f}'.format
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

In [82]:
# stata intermediate data outputs
full_data_stata = pd.read_stata("../../../stata_files/full_data.dta")
before_mcp_stata = pd.read_stata("../../../stata_files/before_mcp_matrix.dta")
rca_stata = pd.read_stata("../../../stata_files/mcp_rca_matrix.dta")
mcp_stata = pd.read_stata("../../../stata_files/mcp_matrix.dta")
ecomp_stata = pd.read_stata("../../../stata_files/ecomplexity.dta")
reliable_stata = pd.read_stata("../../../stata_files/complexity_reliable_countries.dta")
all_countries_stata = pd.read_stata("../../../stata_files/complexity_all_countries.dta")
allcp_stata = pd.read_stata("../../../stata_files/complexity_all_countries_all_products.dta")
comp_stata = pd.read_stata("../../../stata_files/Complexity_H0_2015.dta")

In [74]:
# python intermediate data outputs
full_data_py = pd.read_parquet("../data/intermediate/H0_2015_complexity_all_countries.parquet")
before_mcp_py = pd.read_parquet("../data/intermediate/H0_2015_before_mcp.parquet")
rca_py = pd.read_parquet("../data/intermediate/H0_2015_mcp_rca.parquet")
mcp_py = pd.read_parquet("../data/intermediate/H0_2015_mcp.parquet")
ecomp_py = pd.read_parquet("../data/intermediate/H0_2015_complexitytest.parquet")
reliable_py = pd.read_parquet("../data/intermediate/H0_2015_reliable_countries.parquet")
all_countries_py = pd.read_parquet("../data/intermediate/H0_2015_all_countries.parquet")
allcp_py = pd.read_parquet("../data/intermediate/H0_2015_all_countries_all_products.parquet")
comp_py = pd.read_parquet("../data/processed/H0_2015_complexity.parquet")

In [47]:
def stata_python_data_review(df_stata, df_py, merge_on, value_checks):
    print(f"stata df shape: {df_stata.shape} py df shape {df_py.shape}")
    merged = df_py.merge(df_stata, on=merge_on, how='outer', suffixes=('_py', '_stata'))
    errors = pd.DataFrame()
    for col in value_checks: 
        print(f"reviewing {col}")
        merged[f'{col}_diff'] = merged[f'{col}_stata'] - merged[f'{col}_py']
        print(f"floating point precision is met: {np.isclose(merged[f'{col}_stata'], merged[f'{col}_py']).all() == True}")
        not_close = merged[~np.isclose(merged[f'{col}_stata'], merged[f'{col}_py'])]
        errors = pd.concat([errors, not_close])
    return merged, errors
    

In [48]:
# full data 
full_data_stata = full_data_stata.drop(columns='year')
full_data_merged, not_close = stata_python_data_review(full_data_stata, full_data_py, ['exporter', 'commoditycode'], ['export_value', 'import_value'])

stata df shape: (289224, 7) py df shape (289224, 7)
reviewing export_value
floating point precision is met: True
reviewing import_value
floating point precision is met: True


In [49]:
# before mcp
before_mcp_stata.columns, before_mcp_py.columns
before_mcp_stata.shape, before_mcp_py.shape
before_mcp_merged, not_close = stata_python_data_review(before_mcp_stata, before_mcp_py, ['exporter', 'commoditycode'], ['export_value'])

stata df shape: (163989, 5) py df shape (163989, 5)
reviewing export_value
floating point precision is met: True


In [50]:
# rca matrix
rename_dict = {"HH": "HH_index"}
rca_stata = rca_stata.rename(columns = rename_dict)
rca_merged, not_close_rca = stata_python_data_review(rca_stata, rca_py, ['exporter', 'commoditycode'], ['export_value', 'rca', 'mcp', 'HH_index'])

stata df shape: (163989, 11) py df shape (163989, 8)
reviewing export_value
floating point precision is met: True
reviewing rca
floating point precision is met: True
reviewing mcp
floating point precision is met: True
reviewing HH_index
floating point precision is met: True


In [51]:
# mcp
rename_dict = {
    'HH': 'HH_index',
    'cumshare': 'cumul_share',
    'ene': 'eff_exporters',
    'c1': 'flag_for_small_share',
    'c2': 'flag_for_few_exporters',
    'c3': 'flag_for_low_ubiquity',
    'call': 'exclude_flag'
}

mcp_stata = mcp_stata.rename(columns = rename_dict)
mcp_cols = ['commoditycode', 'export_value', 'HH_index', 'mcp', 'share',
        'cumul_share', 'eff_exporters', 'flag_for_small_share',
        'flag_for_few_exporters', 'flag_for_low_ubiquity', 'exclude_flag']
mcp_merged = stata_python_data_review(mcp_stata, mcp_py, ['commoditycode'], ['export_value', 'HH_index', 'mcp', 'share', 'exclude_flag'])

stata df shape: (1233, 11) py df shape (1233, 11)
reviewing export_value
floating point precision is met: True
reviewing HH_index
floating point precision is met: True
reviewing mcp
floating point precision is met: True
reviewing share
floating point precision is met: True
reviewing exclude_flag
floating point precision is met: True


In [70]:
# ecomp
ecomp_py.shape, ecomp_stata.shape
ecomp_py.columns, ecomp_stata.columns
ecomp_stata = ecomp_stata.rename(columns={'M': 'mcp'})
ecomp_stata = ecomp_stata.astype({
    'exporter': 'object',
    'commoditycode': 'object',
    'export_value': 'float32',
    'diversity': 'int64',
    'ubiquity': 'int64',
    'mcp': 'int64',
    'eci': 'float64',
    'pci': 'float64',
    'density': 'float64',
    'coi': 'float64',
    'cog': 'float64',
    'rca': 'float32'
})
ecomp_merged, not_close_ecomp = stata_python_data_review(ecomp_stata, ecomp_py, ['exporter', 'commoditycode'], ['export_value', 'diversity', 'ubiquity', 'mcp', 'eci', 'pci', 'density', 'coi', 'cog', 'rca'])

stata df shape: (150822, 14) py df shape (150822, 14)
reviewing export_value
floating point precision is met: True
reviewing diversity
floating point precision is met: True
reviewing ubiquity
floating point precision is met: True
reviewing mcp
floating point precision is met: True
reviewing eci
floating point precision is met: True
reviewing pci
floating point precision is met: False
reviewing density
floating point precision is met: False
reviewing coi
floating point precision is met: False
reviewing cog
floating point precision is met: False
reviewing rca
floating point precision is met: True


In [71]:
not_close_ecomp.columns

Index(['exporter', 'commoditycode', 'export_value_py', 'year', 'diversity_py',
       'ubiquity_py', 'mcp_py', 'eci_py', 'pci_normalized', 'density_py',
       'coi_py', 'cog_py', 'rca_py', 'pci_py', 'export_value_stata',
       'population', 'gdp_pc', 'rca_stata', 'mcp_stata', 'density_stata',
       'eci_stata', 'pci_stata', 'diversity_stata', 'ubiquity_stata',
       'coi_stata', 'cog_stata', 'export_value_diff', 'diversity_diff',
       'ubiquity_diff', 'mcp_diff', 'eci_diff', 'pci_diff', 'density_diff',
       'coi_diff', 'cog_diff', 'rca_diff'],
      dtype='object')

In [72]:
cols = ['year', 'exporter', 'commoditycode', 
          'export_value_py', 'export_value_stata', 'export_value_diff',
          'diversity_py', 'diversity_stata', 'diversity_diff',
          'ubiquity_py', 'ubiquity_stata', 'ubiquity_diff',
          'mcp_py', 'mcp_stata', 'mcp_diff',
          'eci_py', 'eci_stata', 'eci_diff',
          'pci_normalized', 'pci_py', 'pci_stata', 'pci_diff',
          'density_py', 'density_stata', 'density_diff',
          'coi_py', 'coi_stata', 'coi_diff',
          'cog_py', 'cog_stata', 'cog_diff',
          'rca_py', 'rca_stata', 'rca_diff',
          'population', 'gdp_pc']
not_close_ecomp = not_close_ecomp[cols]
ecomp_merged = ecomp_merged[cols]
ecomp_merged[ecomp_merged.pci_diff>0.1]

,year,exporter,commoditycode,export_value_py,export_value_stata,export_value_diff,diversity_py,diversity_stata,diversity_diff,ubiquity_py,ubiquity_stata,ubiquity_diff,mcp_py,mcp_stata,mcp_diff,eci_py,eci_stata,eci_diff,pci_normalized,pci_py,pci_stata,pci_diff,density_py,density_stata,density_diff,coi_py,coi_stata,coi_diff,cog_py,cog_stata,cog_diff,rca_py,rca_stata,rca_diff,population,gdp_pc
0,2015,AGO,0101,0.00000,0.00000,0.00000,8,8,0,15,15,0,0,0,0,-1.11344,-1.11343,0.00000,0.74048,0.00330,0.74048,0.73718,0.00581,0.00585,0.00004,-1.19944,-1.19787,0.00157,0.53474,0.53540,0.00066,0.00000,0.00000,0.00000,28127720.00000,3100.83057
2,2015,AGO,0103,0.00000,0.00000,0.00000,8,8,0,17,17,0,0,0,0,-1.11344,-1.11343,0.00000,1.88472,0.02150,1.88472,1.86322,0.00582,0.00585,0.00003,-1.19944,-1.19787,0.00157,0.57455,0.57052,-0.00403,0.00000,0.00000,0.00000,28127720.00000,3100.83057
4,2015,AGO,0105,9279.83789,9279.83789,0.00000,8,8,0,33,33,0,0,0,0,-1.11344,-1.11343,0.00000,0.91110,0.00602,0.91110,0.90508,0.00591,0.00594,0.00002,-1.19944,-1.19787,0.00157,0.34266,0.34172,-0.00094,0.00149,0.00149,0.00000,28127720.00000,3100.83057
6,2015,AGO,0201,5859.69287,5859.69287,0.00000,8,8,0,28,28,0,0,0,0,-1.11344,-1.11343,0.00000,0.57829,0.00072,0.57828,0.57756,0.00732,0.00736,0.00003,-1.19944,-1.19787,0.00157,0.24778,0.24735,-0.00043,0.00013,0.00013,0.00000,28127720.00000,3100.83057
8,2015,AGO,0203,111847.07812,111847.08594,0.00781,8,8,0,17,17,0,0,0,0,-1.11344,-1.11343,0.00000,2.24904,0.02729,2.24903,2.22174,0.00543,0.00545,0.00002,-1.19944,-1.19787,0.00157,0.88086,0.87724,-0.00362,0.00209,0.00209,-0.00000,28127720.00000,3100.83057
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150815,2015,ZWE,9618,0.00000,0.00000,0.00000,85,85,0,12,12,0,0,0,0,-0.79126,-0.79126,0.00000,1.91101,0.02192,1.91100,1.88908,0.05087,0.05113,0.00025,-0.91809,-0.91857,-0.00048,0.74727,0.74374,-0.00353,0.00000,0.00000,0.00000,14154937.00000,1410.32922
150816,2015,ZWE,9701,139873.84375,139873.84375,0.00000,85,85,0,5,5,0,0,0,0,-0.79126,-0.79126,0.00000,4.97747,0.07068,4.97745,4.90677,0.03933,0.03969,0.00036,-0.91809,-0.91857,-0.00048,0.93843,0.90353,-0.03490,0.04541,0.04541,0.00000,14154937.00000,1410.32922
150817,2015,ZWE,9702,1854.63196,1854.63196,0.00000,85,85,0,8,8,0,0,0,0,-0.79126,-0.79126,0.00000,3.22619,0.04283,3.22618,3.18334,0.05890,0.05939,0.00049,-0.91809,-0.91857,-0.00048,0.80128,0.78334,-0.01794,0.02096,0.02096,0.00000,14154937.00000,1410.32922
150818,2015,ZWE,9703,1329172.25000,1329172.25000,0.00000,85,85,0,11,11,0,1,1,0,-0.79126,-0.79126,0.00000,0.85515,0.00513,0.85514,0.85002,0.07952,0.07313,-0.00638,-0.91809,-0.91857,-0.00048,0.00000,0.00000,0.00000,2.08140,2.08140,-0.00000,14154937.00000,1410.32922


In [75]:
# reliable country set
reliable_stata.columns, reliable_py.columns
rename_stata = {"rca1": "rca_reliable", "M": "mcp", "eci1": "eci_reliable", "pci1": "pci_reliable", "density1": "density"}
reliable_stata = reliable_stata.rename(columns=rename_stata)
reliable_merged, not_close_reliable = stata_python_data_review(reliable_stata, reliable_py, ['exporter', 'commoditycode'], ['eci_reliable', 'pci_reliable', 'rca_reliable'])

stata df shape: (150822, 14) py df shape (150822, 20)
reviewing eci_reliable
floating point precision is met: True
reviewing pci_reliable
floating point precision is met: True
reviewing rca_reliable
floating point precision is met: True


In [84]:
all_countries_stata.head()

,exporter,commoditycode,export_value,population,gdp_pc,rca1,M,density1,eci1,pci1,diversity,ubiquity,coi,cog
0,AGO,0101,0.00000,28127720.00000,3100.83057,0.00000,0.00000,0.00585,-1.11343,-0.13481,8.00000,15.00000,-1.19787,0.53540
1,AGO,0102,1114.02429,28127720.00000,3100.83057,0.00006,0.00000,0.00884,-1.11343,-0.64367,8.00000,41.00000,-1.19787,-0.05637
2,AGO,0103,0.00000,28127720.00000,3100.83057,0.00000,0.00000,0.00585,-1.11343,0.49645,8.00000,17.00000,-1.19787,0.57052
3,AGO,0104,0.00000,28127720.00000,3100.83057,0.00000,0.00000,0.01076,-1.11343,-1.41822,8.00000,21.00000,-1.19787,-0.18328
4,AGO,0105,9279.83789,28127720.00000,3100.83057,0.00149,0.00000,0.00594,-1.11343,-0.04068,8.00000,33.00000,-1.19787,0.34172


In [85]:
all_countries_py.head()

,exporter,commoditycode,export_value,import_value,rca,mcp,pci_reliable,pci_nonnorm,eci,prody,expy
0,ABW,0101,14128.37402,25784.36328,0.21492,0,-0.13481,0.00330,0.80134,33575.17847,16663.02084
1,ABW,0104,0.00000,5445.36768,0.00000,0,-1.41822,-0.03369,0.80134,2892.97447,16663.02084
2,ABW,0105,123889.79688,285076.37500,1.69645,1,-0.04068,0.00602,0.80134,17629.68156,16663.02084
3,ABW,0106,8595.79590,226926.75000,0.28496,0,-1.41195,-0.03351,0.80134,6035.35621,16663.02084
4,ABW,0201,0.00000,6221719.00000,0.00000,0,-0.22429,0.00072,0.80134,16488.96492,16663.02084


In [78]:
# all_countries
all_countries_stata = all_countries_stata.rename(columns={"eci2": "eci", "rca2": "rca", "pci1":"pci_reliable"})
comp_merged, errors = stata_python_data_review(all_countries_stata, all_countries_py, ['exporter', 'commoditycode'], ['rca', 'eci', 'prody', 'pci_reliable'])

stata df shape: (265356, 14) py df shape (265356, 11)
reviewing rca
floating point precision is met: True
reviewing eci
floating point precision is met: True
reviewing prody
floating point precision is met: True
reviewing pci_reliable
floating point precision is met: False


In [80]:
errors.sort_values(by="pci_reliable_stata")

,exporter,commoditycode,export_value_py,import_value,rca_py,mcp,pci_reliable_py,pci_nonnorm,eci_py,prody_py,expy_py,export_value_stata,eci_stata,rca_stata,expy_stata,prody_stata,rca1,density1,eci1,pci_reliable_stata,coi,cog,tagc,rca_diff,eci_diff,prody_diff,pci_reliable_diff
0,ABW,0101,14128.37402,25784.36328,0.21492,0,-0.13481,0.00330,0.80134,33575.17847,16663.02084,14128.37402,0.80134,0.21492,16663.02148,33575.17969,NaN,NaN,NaN,NaN,NaN,NaN,1,0.00000,-0.00000,0.00121,NaN
1,ABW,0104,0.00000,5445.36768,0.00000,0,-1.41822,-0.03369,0.80134,2892.97447,16663.02084,0.00000,0.80134,0.00000,16663.02148,2892.97461,NaN,NaN,NaN,NaN,NaN,NaN,0,0.00000,-0.00000,0.00014,NaN
2,ABW,0105,123889.79688,285076.37500,1.69645,1,-0.04068,0.00602,0.80134,17629.68156,16663.02084,123889.79688,0.80134,1.69645,16663.02148,17629.68164,NaN,NaN,NaN,NaN,NaN,NaN,0,0.00000,-0.00000,0.00008,NaN
3,ABW,0106,8595.79590,226926.75000,0.28496,0,-1.41195,-0.03351,0.80134,6035.35621,16663.02084,8595.79590,0.80134,0.28496,16663.02148,6035.35645,NaN,NaN,NaN,NaN,NaN,NaN,0,0.00000,-0.00000,0.00024,NaN
4,ABW,0201,0.00000,6221719.00000,0.00000,0,-0.22429,0.00072,0.80134,16488.96492,16663.02084,0.00000,0.80134,0.00000,16663.02148,16488.96484,NaN,NaN,NaN,NaN,NaN,NaN,0,0.00000,-0.00000,-0.00007,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260815,WSM,2927,0.00000,0.00000,0.00000,0,0.33966,0.01698,0.31930,14051.23480,12085.30178,0.00000,0.31930,0.00000,12085.30176,14051.23535,NaN,NaN,NaN,NaN,NaN,NaN,0,0.00000,0.00000,0.00055,NaN
260816,WSM,8106,0.00000,0.00000,0.00000,0,-0.65182,-0.01160,0.31930,21954.72797,12085.30178,0.00000,0.31930,0.00000,12085.30176,21954.72852,NaN,NaN,NaN,NaN,NaN,NaN,0,0.00000,0.00000,0.00055,NaN
260817,WSM,8109,0.00000,0.00000,0.00000,0,0.72278,0.02802,0.31930,24032.95931,12085.30178,0.00000,0.31930,0.00000,12085.30176,24032.96094,NaN,NaN,NaN,NaN,NaN,NaN,0,0.00000,0.00000,0.00163,NaN
260818,WSM,8601,0.00000,0.00000,0.00000,0,1.13072,0.03978,0.31930,15360.58005,12085.30178,0.00000,0.31930,0.00000,12085.30176,15360.58105,NaN,NaN,NaN,NaN,NaN,NaN,0,0.00000,0.00000,0.00101,NaN


In [86]:
allcp_py.head()

,exporter,commoditycode,export_value,gdp_pc,import_value,rca,mcp,eci,pci,prody
0,ABW,0101,14128.37402,28421.38700,25784.36328,0.21674,0,0.80134,0.42815,35436.47090
1,ABW,0104,0.00000,28421.38700,5445.36768,0.00000,0,0.80134,-0.37161,1271.80519
2,ABW,0105,123889.79688,28421.38700,285076.37500,1.71085,1,0.80134,0.51551,17453.02637
3,ABW,0106,8595.79590,28421.38700,226926.75000,0.28738,0,0.80134,-0.21043,3423.61475
4,ABW,0201,0.00000,28421.38700,6221719.00000,0.00000,0,0.80134,0.41193,16228.32363


In [88]:
allcp_stata.head()

,exporter,commoditycode,export_value,rca3,rca2,rca1,pci3,pci1,eci2,eci1,import_value,gdp_pc,M,density3,prody3,opportunity_value,opportunity_gain,expy,prody,density1,coi,cog,tagc,tagp
0,ABW,0101,14128.37402,0.21674,0.21492,NaN,-0.25841,NaN,0.80134,NaN,25784.36328,28421.38700,0,0.08592,35436.46875,-0.00337,0.00127,16663.02148,33575.17969,NaN,NaN,NaN,1,1
1,ABW,0102,0.00000,0.00000,0.00000,NaN,-0.83901,NaN,0.80134,NaN,0.00000,28421.38700,0,0.09023,5726.13525,-0.00337,-0.00081,16663.02148,10189.50977,NaN,NaN,NaN,0,1
2,ABW,0103,0.00000,0.00000,0.00000,NaN,0.69662,NaN,0.80134,NaN,0.00000,28421.38700,0,0.07383,31966.99805,-0.00337,0.00201,16663.02148,34897.09766,NaN,NaN,NaN,0,1
3,ABW,0104,0.00000,0.00000,0.00000,NaN,-1.95191,NaN,0.80134,NaN,5445.36768,28421.38700,0,0.07873,1271.80530,-0.00337,-0.00117,16663.02148,2892.97461,NaN,NaN,NaN,0,1
4,ABW,0105,123889.79688,1.71085,1.69645,NaN,-0.07342,NaN,0.80134,NaN,285076.37500,28421.38700,1,0.09417,17453.02734,-0.00337,0.00000,16663.02148,17629.68164,NaN,NaN,NaN,0,1


In [30]:
# all_countries, all products
allcp_py.columns, allcp_stata.columns
# allcp_py.dtypes, allcp_stata.dtypes
# allcp_stata = allcp_stata.rename(columns={"pci3": "pci", "rca3": "rca", "density3": "density", "prody3":"prody", "M":"mcp"})
allcp_stata = allcp_stata[['exporter', 'commoditycode', 'export_value', 'gdp_pc', 'import_value','rca','pci', 'prody']]
# allcp_py['prody'] = allcp_py['prody'].astype('float32')
allcp_merged = stata_python_data_review(allcp_stata, allcp_py, ['exporter', 'commoditycode'], ['pci', 'rca'])

stata df shape: (289224, 9) py df shape (289224, 10)
reviewing pci
floating point precision is met: False
  exporter commoditycode  export_value_py  gdp_pc_py  import_value_py  rca_py  \
0      ABW          0101          14128.4    28421.4          25784.4     0.2   
1      ABW          0104              0.0    28421.4           5445.4     0.0   
2      ABW          0105         123889.8    28421.4         285076.4     1.7   
3      ABW          0106           8595.8    28421.4         226926.8     0.3   
4      ABW          0201              0.0    28421.4        6221719.0     0.0   

   mcp  eci  pci_py  prody_py  export_value_stata  gdp_pc_stata  \
0    0  0.8     0.4   35436.5             14128.4       28421.4   
1    0  0.8    -0.4    1271.8                 0.0       28421.4   
2    1  0.8     0.5   17453.0            123889.8       28421.4   
3    0  0.8    -0.2    3423.6              8595.8       28421.4   
4    0  0.8     0.4   16228.3                 0.0       28421.4   

   i

In [32]:
allcp_merged.rca_diff.sum(), allcp_merged.pci_diff.sum()#, allcp_merged.prody_diff.sum()

allcp_merged.sort_values(by='pci_diff')

,exporter,commoditycode,export_value_py,gdp_pc_py,import_value_py,rca_py,mcp,eci,pci_py,prody_py,export_value_stata,gdp_pc_stata,import_value_stata,rca_stata,pci_stata,prody_stata,prody_stata,pci_diff,rca_diff
255732,TKL,2609,0.0,NaN,0.0,0.0,0,0.4,-1.0,1283.9,0.0,NaN,0.0,0.0,-3.3,1283.9,2244.6,-2.3,0.0
249552,TCA,2609,0.0,25783.3,0.0,0.0,0,0.2,-1.0,1283.9,0.0,25783.3,0.0,0.0,-3.3,1283.9,2244.6,-2.3,0.0
239664,SUR,2609,0.0,8907.8,0.0,0.0,0,-0.7,-1.0,1283.9,0.0,8907.8,0.0,0.0,-3.3,1283.9,2244.6,-2.3,0.0
92580,FLK,2609,0.0,NaN,0.0,0.0,0,0.3,-1.0,1283.9,0.0,NaN,0.0,0.0,-3.3,1283.9,2244.6,-2.3,0.0
35724,BLR,2609,0.0,5967.1,0.0,0.0,0,0.6,-1.0,1283.9,0.0,5967.1,0.0,0.0,-3.3,1283.9,2244.6,-2.3,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284266,WSM,XXXX,0.0,4048.5,0.0,NaN,0,0.3,NaN,0.0,0.0,4048.5,0.0,NaN,-1.2,0.0,NaN,NaN,NaN
285502,YEM,XXXX,0.0,1488.4,0.0,NaN,0,-1.1,NaN,0.0,0.0,1488.4,0.0,NaN,-1.2,0.0,NaN,NaN,NaN
286738,ZAF,XXXX,0.0,6204.9,0.0,NaN,0,0.2,NaN,0.0,0.0,6204.9,0.0,NaN,-1.2,0.0,NaN,NaN,NaN
287974,ZMB,XXXX,0.0,1307.9,0.0,NaN,0,-0.4,NaN,0.0,0.0,1307.9,0.0,NaN,-1.2,0.0,NaN,NaN,NaN


In [34]:
# complexity set
comp_merged, not_close_comp = stata_python_data_review(comp_stata, comp_py, ['year', 'exporter', 'commoditycode'], ['rca', 'mcp', 'eci', 'pci', 'oppval', 'oppgain', 'distance', 'prody', 'expy'])

stata df shape: (289224, 16) py df shape (288522, 16)
reviewing rca
floating point precision is met: False
reviewing mcp
floating point precision is met: False
reviewing eci
floating point precision is met: False
reviewing pci
floating point precision is met: False
reviewing oppval
floating point precision is met: False
reviewing oppgain
floating point precision is met: False
reviewing distance
floating point precision is met: False
reviewing prody
floating point precision is met: False
reviewing expy
floating point precision is met: False


In [ ]:
not_close_comp.head()

In [ ]:
reliabl